```arduino
flask-sqlite3-test/
├── instance/
│   └── test.db
├── templates/
│   ├── index.html
│   ├── group_modal.html
│   └── user_modal.html
├── static/
│   └── js/
│       ├── group_modal.js
│       ├── user_modal.js
│       └── index.js
├── db_init.py
└── app.py
```

In [5]:
from datetime import datetime, timezone

# 确保日期格式为 <class 'datetime.datetime'>
def date_for_sqlite(meta_date:str) -> datetime:
    if meta_date == '':
        new_date = datetime.now(timezone.utc)
    else:
        new_date = datetime.strptime(meta_date, '%Y-%m-%d')
    return new_date

date_for_sqlite('2024-05-18')

datetime.datetime(2024, 5, 18, 0, 0)

In [5]:
import pandas as pd
from app import app, db
from app import Group, User

from sqlalchemy.inspection import inspect
from sqlalchemy.exc import OperationalError

with app.app_context():

    try:
        # 读取表格内容为 list，其中的元素都为各自对应模型的实例
        group_list = Group.query.all()
        user_list = User.query.all()
        group_columns = inspect(Group).columns
        user_columns = inspect(User).columns
    except OperationalError as e:
        print(f"OperationalError: {e}")

print(f"{group_columns.id.key}|{group_columns.name.key}|{group_columns.created_on.key}|{group_columns.first_register.key}|{group_columns.last_register.key}")
for group in group_list:
    print(
        f"{group.id}|{group.name}|{group.created_on}|{group.first_register}|{group.last_register}"
    )

print(f"{user_columns.id.key}|{user_columns.name.key}|{user_columns.registered_on.key}|{user_columns.group_id.key}")
for user in user_list:
    print(
        f"{user.id}|{user.name}|{user.registered_on}|{user.group_id}"
    )


id|name|created_on|first_register|last_register
1|Group A|2024-04-15 21:43:02.591253|2024-06-05|2024-06-05
2|Group B|2024-05-03 21:43:02.591253|2024-06-06|2024-06-10
3|Group C|2024-05-07 21:43:02.591253|None|None
id|name|registered_on|group_id
1|Peter|2024-06-10 21:43:02.599258|2
2|Stewie|2024-06-05 21:43:02.599258|1
3|Brian|2024-06-10 21:43:02.599258|0
4|Lois|2024-06-06 21:43:02.599258|2
5|Meg|2024-06-08 21:43:02.599258|0
6|Chris|2024-06-08 21:43:02.599258|0


In [ ]:
user_data = [{'id': user.id, 
              'name': user.name, 
              'registered_on': user.registered_on,
              'group_id': user.group_id} 
              for user in user_list]
# group_id 列因为可能有空值None存在导致被识别为浮点数NaN，进而导致整列变成浮点数
user_df = pd.DataFrame(user_data)
user_df['group_id'] = user_df['group_id'].astype(pd.Int64Dtype())

In [ ]:
user_df

In [ ]:
from datetime import datetime, timezone, timedelta

# type <datetime.date>
A = datetime.strptime('2024-06-06', '%Y-%m-%d').date() + timedelta(days=1)
B = datetime(2024, 6, 6, tzinfo=timezone.utc) + timedelta(days=1)
A == B